In [2]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
from lyricsgenius import Genius
from tqdm.autonotebook import tqdm as notebook_tqdm

/tmp/ipykernel_3340/2948612796.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
genius_token = os.getenv('GENIUS_ACCESS_TOKEN')

In [4]:
genius = Genius(genius_token)

In [5]:
artist = genius.search_artist("Huncho Jack", max_songs=3, sort="popularity", allow_name_change=True)

Searching for songs by Huncho Jack...

Changing artist name to 'HUNCHO JACK'
Song 1: "Dubai Shit"
Song 2: "Modern Slavery"
Song 3: "Black & Chinese"

Reached user-specified song limit (3).
Done. Found 3 songs.


In [10]:
artist.save_lyrics(filename='lyrics/hunchjack')

Wrote lyricshunchjack.json.


In [7]:
# load pickle file
artists_dict = pd.read_pickle('../dict_featuresALL-modified.pickle')

In [10]:
artist_set = set()

for artist in artists_dict:
    artist_set.add(artist)
    for feat in artists_dict[artist]:
        artist_set.add(feat)
len(artist_set)

16783

In [13]:
for artist in artist_set:
    try:
        print(artist)
        genius_artist = genius.search_artist(artist[0], max_songs=3, sort="popularity", allow_name_change=True)
        genius_artist.save_lyrics(f'{artist[0]}')
    
    except Exception as e:
        print(e)

('!llmind', '0bw7sLDsNQTScMpmsNFukJ')
Searching for songs by !llmind...

Song 1: "Take A Break (Interlude)"
"Check the Credits" is not valid. Skipping.
Song 2: "March"
Song 3: "Po Po Comin"

Reached user-specified song limit (3).
Done. Found 3 songs.
Wrote llmind.json.
('Young Smoke', '3XbEIIeUCZdEDGyTX5pjVD')
Searching for songs by Young Smoke...

Song 1: "Days By Da Rabba"
Song 2: "Smokey World"
Song 3: "4 Tha Streetz"

Reached user-specified song limit (3).
Done. Found 3 songs.
Wrote Young Smoke.json.
('Clyde Carson', '0FJpfR5Bq5EBl2xi0LAp1v')
Searching for songs by Clyde Carson...

Song 1: "Slow Down"
Song 2: "Slow Down (Remix)"
Song 3: "Tuk Spot"

Reached user-specified song limit (3).
Done. Found 3 songs.
Wrote Clyde Carson.json.
('D72', '6LwMT7S1AQrvSORZshZ9gt')
Searching for songs by D72...

Song 1: "Switch Play"
Song 2: "Switch Play (Extended Mix)"
Done. Found 2 songs.
Wrote D72.json.
('Mela Lee', '5eOlFIMqTRkgL6kd06usSw')
Searching for songs by Mela Lee...

Done. Found 0 song

KeyboardInterrupt: 